In [ ]:
# Install necessary libraries
!pip install transformers datasets librosa torch

import torch
import librosa
from transformers import ClapProcessor, ClapModel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# Load CLAP processor and model
processor = ClapProcessor.from_pretrained("laion/clap-htsat-unfused")
model = ClapModel.from_pretrained("laion/clap-htsat-unfused")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/5.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/615M [00:00<?, ?B/s]

In [ ]:
import os
input_dir = "/content/Data"
output_dir = "/content/Data"
os.makedirs(output_dir, exist_ok=True)

for file in os.listdir(input_dir):
    if file.endswith((".mp3", ".wav", ".flac")):
        input_path = os.path.join(input_dir, file)
        output_path = os.path.join(output_dir, f"{os.path.splitext(file)[0]}.wav")
        !ffmpeg -i "{input_path}" -ar 48000 -acodec pcm_s16le "{output_path}"

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
# ---- Load and Preprocess Audio ----
audio_path = "/content/Data/generated_music (3).wav"  # Replace with your audio file
audio, sr = librosa.load(audio_path, sr=48000, duration=10)  # Load 30 seconds, 48kHz (CLAP default)

In [ ]:
# Preprocess audio for CLAP
audio_inputs = processor(audios=audio, return_tensors="pt")

# Extract audio embeddings
with torch.no_grad():
    audio_embeddings = model.get_audio_features(**audio_inputs)

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [ ]:
# ---- Define Prompts (Text) ----
prompts = ["An energetic a beat pop track with drumpad and guitars"]

# ---- Generate Text Embeddings ----
def get_text_embeddings(prompts):
    embeddings = {}
    for text in prompts:
        inputs = processor(text=text, return_tensors="pt")
        with torch.no_grad():
            embeddings[text] = model.get_text_features(**inputs)
    return embeddings

text_embeddings = get_text_embeddings(prompts)

# ---- Compute Cosine Similarity ----
def compute_similarity(audio_embeds, text_embeds):
    similarities = {
        text: torch.nn.functional.cosine_similarity(audio_embeds, text_embeds[text]).item()
        for text in text_embeds
    }
    return similarities

similarity_results = compute_similarity(audio_embeddings, text_embeddings)

# ---- Get the Closest Match ----
best_match = max(similarity_results, key=similarity_results.get)
best_score = similarity_results[best_match]

# ---- Display Results ----
print("\nPredicted Description:")
print(f"{best_match} (Similarity: {best_score:.2f})")


Predicted Description:
An energetic a beat pop track with drumpad and guitars (Similarity: 0.45)
